In [2]:
import os            # для работы с операционной системой
import pickle as pkl # для сохранения и загрузки переменных
import pandas as pd  # для работы с таблицами
import numpy  as np  # для работы с массивами
import re            # для работы с регулярными выражениями
import zlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.text import Tokenizer

#from sklearn.feature_extraction.text import TfidfVectorizer
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [74]:
# Загрузка датафрейма
# with open( 'total_transcriptions_text_df.pkl', 'rb') as f:
#    df = pkl.load(f)
#f_dir = '/content/drive/MyDrive/Traineeships/media108/'
f_dir = 'c:/Users/vokulovskiy/Documents/python/media108/'
f_name = 'df_new_good.xlsx'
f_name_mod = 'df_new_good_mod1.xlsx'
f_name_mod_csv = 'df_new_good_mod1.csv'
#df = pd.read_excel(f_dir+f_name_mod)
df = pd.read_csv(f_dir+f_name_mod_csv, sep=';',encoding='cp1251')
#df = pd.read_csv(f_name)
#df.to_excel('f_dir+f_name', index=False)

# Подготовка данных

In [35]:
# оставим поля Name, Текст_whisper, Дата и время, Длительность звонка, Номер обращения, Признак, Теги
new_col = {'Текст_whisper':'text','Дата и время':'date_time','Длительность звонка':'length','Номер обращения':'ref_num','Name':'name','Теги':'tags','Признак':'target'}
df.rename(columns = new_col, inplace=True)
df = df.loc[:,list(new_col.values())]
#Удаляем из Текст_whisper все символы кроме русских букв и цифр
df.text = df.text.apply(lambda s: re.sub("[^А-Яа-я0-9 ]", "", s.lower()).strip())
# Заменим буквы 'ё' на 'е'
df.text = df.text.str.replace('ё', 'е', regex=False)
# удаляем строки с пустым текстом
df = df[df.text.str.len()>0]
# поменяем тип столбца с датами
df.date_time = pd.to_datetime(df.date_time, format='%Y-%m-%d %H:%M:%S')
# Сколько целевых звонков длиной меньше х сек?
for x in range(30,76,5):
    print(f'Целевых звонков с длительностью < {x} сек: {df[(df.length<x)&(df.target==1)].target.count()}, всего звонков {df[df.length<x].target.count()}')
# Целевых звонков с длительностью < 30 сек: 2, всего звонков 1737
# Удаляем звонки с длительностью < 30 сек
df = df[(df.length>30)]
print(df.target.value_counts())
"""
0    2528
1    2263"""

Целевых звонков с длительностью < 30 сек: 0, всего звонков 0
Целевых звонков с длительностью < 35 сек: 11, всего звонков 122
Целевых звонков с длительностью < 40 сек: 22, всего звонков 261
Целевых звонков с длительностью < 45 сек: 40, всего звонков 383
Целевых звонков с длительностью < 50 сек: 53, всего звонков 516
Целевых звонков с длительностью < 55 сек: 68, всего звонков 628
Целевых звонков с длительностью < 60 сек: 81, всего звонков 759
Целевых звонков с длительностью < 65 сек: 95, всего звонков 867
Целевых звонков с длительностью < 70 сек: 109, всего звонков 966
Целевых звонков с длительностью < 75 сек: 123, всего звонков 1090
target
0    2528
1    2263
Name: count, dtype: int64


'\n0    2528\n1    2263'

In [ ]:
'''
# Добавление колонки "Признак" и заполнение ее данными на основе колоки "Теги"
#df.insert(7,'target1',value=np.NaN)
list_teg = df.tags.str.strip().tolist()
list_priz = []
for i in range(len(list_teg)):
    list_priz.append(1 if (re.search(r'\bПервичный целевой\b',list_teg[i],re.I) != None or re.search(r'\bЦелевой_М108',list_teg[i]) != None) else 0)
    if (re.search(r'\b[Нн]ецелевой\b',list_teg[i],re.I) != None or re.search(r'\b[нН]ецелевой_',list_teg[i]) != None):
        list_priz[i] = 0
df.target1 = list_priz
'''

In [38]:
#df.sample(5)
#df.info()
#df.nunique()
df.to_excel(f_dir+f_name_mod, index=False)
#df.target.value_counts()

In [ ]:
'''
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf_features = vectorizer.fit_transform(df.text).toarray()
tfidf_features.shape
# Мешок слов до обработки pymorphy2: 66787 после 66025 нет ни какого смысла в этой операции
'''

In [ ]:
'''
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
df['text_norm'] = df.text.apply(lambda x: ' '.join([morph.parse(s)[0].normal_form for s in x.split()]))
df.loc[1:2,['text','text_norm']]
'''

BoW wisper text

In [75]:
words_txt = 35000
tokenizer_txt = Tokenizer(num_words=words_txt, filters='i!"#$%&()*+,-–—/…:.;<=>?@[\\]^`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer_txt.fit_on_texts(list(df.text))
print(max(tokenizer_txt.word_index.values()))

66015
OrderedDict([('здравствуйте', 9980), ('вы', 24829), ('позвонили', 2413), ('в', 54996), ('группу', 1564), ('компании', 3407), ('картрофт', 532), ('благодарим', 1497), ('вас', 23473), ('за', 6407), ('звонок', 2728), ('пожалуйста', 9395), ('дождитесь', 2171), ('ответа', 2186), ('оператора', 1490), ('группа', 1203), ('меня', 10328), ('зовут', 6376), ('елена', 1004), ('интересует', 2434), ('спедлайнер', 1), ('я', 40295), ('сейчас', 13088), ('соединю', 894), ('с', 22417), ('одним', 66), ('упрадашником', 1), ('он', 3309), ('жестко', 6), ('консультирует', 84), ('как', 15532), ('ирина', 431), ('александровна', 18), ('благодарю', 390), ('оставайтесь', 1003), ('на', 36321), ('линии', 1881), ('дарья', 373), ('какие', 2125), ('вопросы', 1303), ('по', 20963), ('проекту', 594), ('ходланд', 2), ('слушаю', 917), ('добрый', 4970), ('день', 5313), ('смотрите', 3234), ('одна', 984), ('квартира', 4914), ('41', 356), ('квадрат', 156), ('20', 1406), ('пейсов', 1), ('у', 31648), ('нас', 16090), ('есть',

In [119]:
# Получение матрицы BOW для колонки с текстом 
bow_txt = tokenizer_txt.texts_to_matrix(df.text.tolist())
# Создаем сжатую колонку с BoW
df['text_bow'] = None
for i in range(len(bow_txt)):
    df.loc[i,'text_bow'] = zlib.compress(''.join([str(int(y)) for y in bow_txt[i]]).encode('utf-8'))
bow_txt.shape

(4791, 35000)

[BERT](https://habr.com/ru/articles/562064/)

In [ ]:
! pip install transformers sentencepiece
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)

In [ ]:
# Создаем колонку с эмбеддингами разговоров, размерность 312
df['text_emb']=df.text.apply(lambda x: ','.join([str(y) for y in embed_bert_cls(x, model, tokenizer)]))

In [83]:
# Сохраняемся
# df.to_excel('f_dir+f_name_mod', index=False)
df.to_csv(f_dir+f_name_mod_csv, index=False,sep=';',encoding='cp1251')
with open(f_dir+'token_encod.pkl', 'wb') as f:
    pkl.dump([tokenizer_txt], f)

# Извлечение данных из CSV файла

In [ ]:
# Load from file 
#with open(pkl_filename, 'rb') as file: 
#pickle_model = pickle.load(file) 
#f_dir = '/content/drive/MyDrive/Traineeships/media108/'
f_dir = 'c:/Users/vokulovskiy/Documents/python/media108/'
f_name_mod_csv = 'df_new_good_mod1.csv'
df = pd.read_csv(f_dir+f_name_mod_csv, sep=';')
#Обратное преобразование, загрузка эмбеддингов/Bow из .csv
x1 = []
x2 = []
for i,row in df.iterrows():
    x1.append([float(j) for j in row.text_emb.split(',')])
    x2.append(np.array(list(zlib.decompress(eval(row.text_bow)).decode('utf-8')), dtype=float))
X1 = np.array(x1) # Embeddings
X2 = np.array(x2) # Bow
y = df.target.to_numpy()
print(X1.shape,X2.shape,y.shape)

In [ ]:
# Делим на выборки
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, train_size=0.75, test_size=0.25, random_state=313187, stratify=y)
X1_val, X1_test, y1_val, y1_test = train_test_split(X1_test, y1_test, train_size=0.75, test_size=0.25, random_state=545212, stratify=y1_test)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, train_size=0.75, test_size=0.25, random_state=313187, stratify=y)
X2_val, X2_test, y2_val, y2_test = train_test_split(X2_test, y2_test, train_size=0.75, test_size=0.25, random_state=545212, stratify=y2_test)
print(y1_val.shape, y2_val.shape, sum(y1_val==y2_val))
print(y1_test.shape, y2_test.shape, sum(y1_test==y2_test))

# ML

In [13]:
# LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
pred_train = model.predict(X_train)
pred_val = model.predict(X_test)
print('Accuracy_train:',balanced_accuracy_score(y_train, pred_train),'  accuracy_test:', balanced_accuracy_score(y_test, pred_val))


Accuracy_train: 0.8499057918592723   accuracy_test: 0.8476932591218306


In [14]:

# DecisionTree
dt = DecisionTreeClassifier(max_depth=5, min_samples_leaf=9, random_state=10)
dt.fit(X_train, y_train)
pred_train = dt.predict(X_train)
pred_val = dt.predict(X_test)
print('Accuracy_train:',balanced_accuracy_score(y_train, pred_train), '  accuracy_test', balanced_accuracy_score(y_test, pred_val))


Accuracy_train: 0.856761528746766   accuracy_test 0.8013605442176871


In [15]:
# RandomForest

model = RandomForestClassifier(min_samples_leaf=3, max_depth=8, n_estimators=13, random_state=10)
model.fit(X_train, y_train)
pred_train = model.predict(X_train)
pred_val = model.predict(X_test)
print('Accuracy_train:',balanced_accuracy_score(y_train, pred_train),'  accuracy_test:', balanced_accuracy_score(y_test, pred_val))


Accuracy_train: 0.917906767318835   accuracy_test: 0.8431663574520717


In [16]:
#XGboost
from xgboost import XGBClassifier
from IPython.display import clear_output
x_val, x_test1,  y_val, y_test1 = train_test_split(X_test, y_test, test_size=0.25, random_state=15)
model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.09, early_stopping_rounds=5)
model.fit(X_train, y_train, eval_set=[(X_train, y_train,), (x_val, y_val)])
pred_train = model.predict(X_train)
pred_val = model.predict(x_val)
pred_test = model.predict(x_test1)
clear_output()
print('Accuracy_train:',balanced_accuracy_score(y_train, pred_train),'  accuracy_val:',balanced_accuracy_score(y_val, pred_val),'  accuracy_test:', balanced_accuracy_score(y_test1, pred_test))


Accuracy_train: 0.9497878102976378   accuracy_val: 0.8531036759850319   accuracy_test: 0.8303807457092129


In [ ]:
!pip install tpot

In [6]:
"""
from tpot import TPOTClassifier

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline.py')
"""

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.852878872100662

Generation 2 - Current best internal CV score: 0.8549554466285985

Generation 3 - Current best internal CV score: 0.8549554466285985

Generation 4 - Current best internal CV score: 0.8549554466285985

Generation 5 - Current best internal CV score: 0.8549554466285985

Best pipeline: GradientBoostingClassifier(RFE(MinMaxScaler(input_matrix), criterion=entropy, max_features=0.15000000000000002, n_estimators=100, step=0.45), learning_rate=0.1, max_depth=10, max_features=0.15000000000000002, min_samples_leaf=14, min_samples_split=17, n_estimators=100, subsample=0.8)
0.845136186770428


Проверим предложенную модель

In [24]:
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

training_features, testing_features, training_target, testing_target = /
            train_test_split(X, y, train_size=0.75, test_size=0.25,random_state=23) #10,23

# Average CV score on the training set was: 0.8549554466285985
exported_pipeline = make_pipeline(
    MinMaxScaler(),
    RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.15000000000000002, n_estimators=100), step=0.45),
    GradientBoostingClassifier(learning_rate=0.1, max_depth=10, max_features=0.15000000000000002, min_samples_leaf=14, min_samples_split=17, n_estimators=100, subsample=0.8)
)

exported_pipeline.fit(training_features, training_target)

pred_train = model.predict(training_features)
pred_val = model.predict(testing_features)
print('Accuracy_train:',balanced_accuracy_score(training_target, pred_train),'  accuracy_test:', balanced_accuracy_score(testing_target, pred_val))


Accuracy_train: 0.9626361916852555   accuracy_test: 0.9594262861315885


In [28]:
model = GradientBoostingClassifier(learning_rate=0.1, max_depth=10, max_features=0.15, min_samples_leaf=14, min_samples_split=17, n_estimators=100, subsample=0.8)
model.fit(X_train, y_train)
pred_train = model.predict(X_train)
pred_val = model.predict(X_test)
print('Accuracy_train:',balanced_accuracy_score(y_train, pred_train),'  accuracy_test:', balanced_accuracy_score(y_test, pred_val))


Accuracy_train: 0.9985405720957385   accuracy_test: 0.8433142374988898
